In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import display, Latex, Markdown, Image, Math
from scipy.integrate import solve_ivp
from scipy.signal import place_poles
import sympy as sp

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

__System__

In [ ]:
KU, AT1, AT2, AS1, AS2, g, hV1, hV2, t = sp.symbols('K_U A_{T1} A_{T2} A_{S1} A_{S2} g h_{V1} h_{V2} t')
s = sp.symbols('s')

In [ ]:
z1 = sp.Function('z_1')
z2 = sp.Function('z_2')
uA = sp.Function('u_A')
y = sp.Function('y')
z = sp.Matrix([[z1(t)], [z2(t)]])

In [ ]:
a1 = AS1 * sp.sqrt(2 * g / (AT1**2 - AS1**2))
a2 = AS2 * sp.sqrt(2 * g / (AT2**2 - AS2**2))

f1 = a1 * sp.sqrt(z1(t) + hV1)
f2 = a2 * sp.sqrt(z2(t) + hV2)
dz1 = - f1 + KU / AT1 * uA(t)
dz2 = - f2 + f1 * AT1 / AT2

dz = sp.Matrix([[dz1], [dz2]])
display(Latex("$\\dot{{\\mathbf{{z}}}} = {}$".format(sp.latex(dz))))

__Flacher Ausgang__

\begin{align*}
    y(t) & = z_2(t)\\
    \dot{y}(t) & = \dot{z}_2(t) = f_2(x_1(t), y(t)) \\
    \ddot{y}(t) & = 
\end{align*}

In [ ]:
u = sp.solve(dz[1].subs({z2(t): y(t)}).diff(t).subs({z1(t).diff(t): dz[0]}).subs({z1(t): x1}) - y(t).diff(t, 2), uA(t))[0]
u

In [ ]:
x1 = sp.solve(dz[1].subs({z2(t): y(t)}) - y(t).diff(t), z1(t))[0]
x1

## Zustandsregler

In [ ]:
imag = Image("../../images/zweitank.png", width=800)
display(imag)

## 2-Tanksystem

- lineares Modell in der Form
    \begin{align*}
        \dot{\tilde{z}}_1(t) & = a_{11} \tilde{z}_1(t) + b \tilde{u}_{\text{A}}(t)\\
        \dot{\tilde{z}}_2(t) & = a_{21} \tilde{z}_1(t) + a_{22} \tilde{z}_2(t) 
    \end{align*}
    mit Ausgang $y(t) = z_2(t)$
- Zustandsdarstellung
    \begin{align*}
        \dot{\mathbf{z}}(t) & = \begin{pmatrix} a_{11} & 0 \\ a_{21} & a_{22} \end{pmatrix} \mathbf{z}(t) + \begin{pmatrix} b\\0 \end{pmatrix} u(t) \\
        y(t) & = \begin{pmatrix} 0 & 1 \end{pmatrix} \mathbf{z}(t)
    \end{align*}
    mit Zustand $\mathbf{z} = \begin{pmatrix} \tilde{z}_1 & \tilde{z}_2 \end{pmatrix}^{\intercal}$ und Eingang $u = \tilde{u}_{\text{A}}(t)$

In [ ]:
time = np.linspace(0, 300, 301)

In [ ]:
x0 = [0.2, 0.15]
A = np.array([[-0.0289, 0], [0.0289, -0.0146]])
b = np.array([[0.0021], [0]])
c = np.array([[0], [1]])

## Steuerbarkeit

In [ ]:
S = np.hstack((b, A.dot(b)))
display(Latex("$\\mathrm{{rang}}\\mathcal{{S}} = \\mathrm{{rang}}{} = {}$".format(sp.latex(sp.Matrix(S)), sp.latex(np.linalg.matrix_rank(S)))))

## Transformation in Regelungsnormalform

- Transformation $\mathbf{x} = \mathbf{T}\mathbf{z}$
- Überführung der Zustandsdarstellung
    \begin{align*}
        \dot{\mathbf{x}}(t) & = \underbrace{\begin{pmatrix} 0 & 1 \\ \alpha_0 & \alpha_1 \end{pmatrix}}_{\mathbf{\tilde{A}}} \mathbf{x}(t) + \underbrace{\begin{pmatrix} 0\\ 1 \end{pmatrix}}_{\mathbf{\tilde{b}}} u(t) \\
        y(t) & = \underbrace{\begin{pmatrix} c_0 & c_1 \end{pmatrix}}_{\mathbf{\tilde{c}}^{\intercal}} \mathbf{x}(t)
    \end{align*}

In [ ]:
q = np.array([0, 1]) @ np.linalg.inv(S)
T = np.vstack((q, q @ A))
display(Latex("$\\mathbf{{T}} = {}$".format(sp.latex(sp.Matrix(T)))))

In [ ]:
tA = T @ A @ np.linalg.inv(T)
tb = T @ b
tc = c.T @ np.linalg.inv(T)
display(Latex("$\\mathbf{{\\tilde{{A}}}} = \\mathbf{{T}} \\mathbf{{A}} \\mathbf{{T}}^{{-1}} = {}$".format(sp.latex(sp.Matrix(tA)))))
display(Latex("$\\mathbf{{\\tilde{{b}}}} = \\mathbf{{T}} \\mathbf{{b}} = {}$".format(sp.latex(sp.Matrix(tb)))))
display(Latex("$\mathbf{{\\tilde{{c}}}}^{{\intercal}} = \\mathbf{{c}}^{{\intercal}} \\mathbf{{T}}^{{-1}} = {}$".format(sp.latex(sp.Matrix(tc)))))

## Systemeigenwerte

In [ ]:
eigA = np.linalg.eig(A)[0]
Math(r's_1 = ' + str(eigA[0]) + r',~s_2 = ' + str(eigA[1]))

## Reglerentwurf

- Festlegung der Pole des geschlossenen System
\begin{align*}
    \det (s - (\tilde{\mathbf{A}} - \tilde{\mathbf{b}} \mathbf{k}^{\intercal})) = s^2 + (\alpha_1 - k_1) s + (\alpha_0 - k_0) = s^2 + p_1 s + p_0
\end{align*}

In [ ]:
poles = np.array([-0.05, -0.06])
pp = place_poles(A, b, poles)
k = pp.gain_matrix[0].reshape((1,2))
eigAr = np.linalg.eig(A - b @ k)[0]
Math(r'\mathbf{k}=' + sp.latex(sp.Matrix(k)) + r'\\' + 'p_1 = ' + str(eigAr[0]) + r',~p_2 = ' + str(eigAr[1]))

## Simulation

In [ ]:
def prozess(t, x, A, b, k):
    u = -k @ x
    dx = A @ x + b @ u

    return dx

In [ ]:
res = solve_ivp(prozess, [time[0], time[-1]], x0,
                t_eval=time, args=(A, b, k))

In [ ]:
plt.close()

fig1, axes10 = plt.subplots(1, 1, figsize=(14,6))

axes10.plot(res.t, res.y[0], label=r"$\tilde{z}_1$")
axes10.plot(res.t, res.y[1], label=r"$\tilde{z}_2$")

axes11 = axes10.twinx()
axes11.plot(res.t, (k @ res.y).T, 'C4')
axes11.set_ylabel(r"$u$ in V", color='C4')
axes11.tick_params(axis='y', colors='C4')

axes10.set_xlabel(r"$t$ in s")
axes10.set_ylabel(r"$\tilde{z}_{\bullet}$ in m")

axes10.grid() 

handlesAx, labelsAx = axes10.get_legend_handles_labels()
fig1.legend([handle for i, handle in enumerate(handlesAx)],
            [label for i, label in enumerate(labelsAx)],
            bbox_to_anchor=(0.125, 0.90, 0.775, .15), loc=3,
            ncol=4, mode="expand", borderaxespad=0., framealpha=0.5)
plt.show()

## Stabilisierung beliebiger Arbeitspunkte

In [ ]:
output = widgets.Output()

with output:
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 6), sharex=True, gridspec_kw={'height_ratios': [1, 1, 1]})

plt.subplots_adjust(wspace=0.2, hspace=0.07)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.set_xlim([0, time[-1]]) 
ax2.set_xlim([0, time[-1]])
ax3.set_xlim([0, time[-1]]) 
ax1.set_ylim([-0.05, 3.55]) 
ax2.set_ylim([-2, 2]) 
ax3.set_ylim([-2, 2]) 
ax1.grid()
ax2.grid()
ax3.grid()
ax1.set_ylabel(r"$y$")
ax2.set_ylabel(r"$e$")
ax3.set_ylabel(r"$u$")
ax3.set_xlabel(r"$t$")

ySys, = ax1.plot([], [], 'C0', label=r'Prozess $y = x_1$')
x2Sys, = ax1.plot([], [], 'C3', label=r'Zustand 2 $x_2$')
yRefSys, = ax1.plot([], [], 'C1', label=r'Referenz $y_\mathrm{r}$')
eSys, = ax2.plot([], [], 'C0', label=r'Fehler $e$')
uFBSys, = ax3.plot([], [], 'C2', label=r'Regler $u$')

handlesAx, labelsAx = ax1.get_legend_handles_labels()

fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.13, 0.94, 0.7675, .15), loc=3,
           ncol=3, mode="expand", borderaxespad=0., framealpha=0.5)

sliderYr = widgets.FloatSlider(value=2,
                               min=0,
                               max=5,
                               step=0.5,
                               description=r'$y_{\mathrm{r}}$')
sliderEig1 = widgets.FloatSlider(value=-3,
                               min=-20,
                               max=-0.01,
                               step=0.1,
                               description=r'$s_1$')
sliderEig2 = widgets.FloatSlider(value=-6,
                               min=-20,
                               max=-0.01,
                               step=0.1,
                               description=r'$s_2$')
sliderX01 = widgets.FloatSlider(value=0,
                               min=-10,
                               max=10,
                               step=0.1,
                               description=r'$x_1(0)$')
sliderX02 = widgets.FloatSlider(value=0,
                               min=-10,
                               max=10,
                               step=0.1,
                               description=r'$x_2(0)$')

def calcSystem(_):
    yr = sliderYr.value
    
    pp = place_poles(A, b, np.array([sliderEig1.value, sliderEig2.value]))
    k = pp.gain_matrix[0].reshape((1,2))
    
    x0 = [sliderX01.value, sliderX02.value]
    res = solve_ivp(linSystemStateContFactor,
                    [time[0], time[-1]],
                    x0,
                    t_eval=time, args=(yr, A, b, c, k))
    
    V = 1 / (c.T @ np.linalg.inv(b @ k - A) @ b)
           
    sigFB = [V * yr - k @ res.y.T[idx + 1, :] for idx, t in enumerate(time[1:])]
    sigE = yr - res.y.T[:, 0]
    uFBSys.set_data(time[1:], sigFB)
    ySys.set_data(time, res.y.T[:, 0])
    x2Sys.set_data(time, res.y.T[:, 1])
    yRefSys.set_data(time[[0, -1]], np.array([yr, yr]))
    eSys.set_data(time, sigE)
    
    uMax = np.max(sigFB)
    ax1.set_ylim(np.min(res.y) - 0.5, np.max(res.y) + 0.5)
    ax2.set_ylim([(np.min(sigE) - 0.2), (np.max(sigE) + 0.2)]) 
    ax3.set_ylim([-(uMax + 1), (uMax + 1)]) 
    
    fig.canvas.draw()

sliderYr.observe(calcSystem, names='value')
sliderEig1.observe(calcSystem, names='value')
sliderEig2.observe(calcSystem, names='value')
sliderX01.observe(calcSystem, names='value')
sliderX02.observe(calcSystem, names='value')

calcSystem(_)

stateContControls = VBox([sliderYr, sliderEig1, sliderEig2])
stateContControls.layout = make_box_layout()
sysControls = VBox([sliderX01, sliderX02])
sysControls.layout = make_box_layout()

controls = HBox([sysControls, stateContControls])
VBox([controls, output], layout=Layout(display='flex', flex_flow='row', justify_content='center', align_items='center'))